随机森林是Bagging的一个扩展变体,RF在以决策树为基学习器构建Bagging集成的基础上,进一步在决策树的训练过程中引入了随机属性选择.

具体来说,传统决策树在选择划分属性时是在当前结点的属性集合中选择一个最优属性,而在RF中,对基决策树的每个结点,先从该结点的属性集合随机选择一个包含k个属性的子集,然后再从这个子集中选择一个人最优属性用于划分.这里的参数k控制了随机性的引入程度:

若k=d则与传统决策树相同,若令 k=1 则是随机选择一个属性用于划分;一般情况下推荐值k=log2d

随机森林中基学习器的多样性不仅来自样本扰动,还来自于属性扰动,这就使得最终集成泛化性能可通过个体学习器之间差异度的增加而进一步提升.

随机森林的起始性能往往相对较差,特别是集成中只包含一个基学习器时.

In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import sys
import warnings

warnings.simplefilter('ignore')

In [2]:
def load_data():
    data = load_digits()
    return data.data,data.target

X,y = load_data()
cross_val_score(DecisionTreeClassifier(),X,y,cv=5)

array([0.78846154, 0.72099448, 0.78272981, 0.81792717, 0.77746479])

In [3]:
DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [4]:
def k_features(n_f):
    return int(np.log2(n_f)) if n_f > 1 else n_f 
k = []
for i in range(10):
    k.append((i,k_features(i)))
k

[(0, 0),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2),
 (8, 3),
 (9, 3)]

1. 首先,从统计的方面来看,由于学习任务的假设空间往往很大,可能有多个假设在训练集上达到同等性能,此时若使用单学习器可能因误选而导致泛化性能不佳,结合多个学习器则会减少这一风险;

2. 从计算的方面来看,学习算法往往会陷入局部极小,有的局部极小点所对应的泛化性能可能很糟糕,而通过多次运行之后进行结合,有的局部极小点所对应的泛化性能可能很糟糕,而通过多次运行之后进行结合,可降低陷入局部极小点的风险;

3. 从表示的方面来看,某些学习任务的真实假设可能不在当前学习算法所考虑的假设空间中,此时若使用单学习器则肯定无效,而通过结合多个学习器,由于相应的假设空间有所扩大,有可能学得更好的近似.

In [5]:
rf = RandomForestClassifier(n_estimators=5)
cross_val_score(rf,X,y,cv=5)

array([0.87087912, 0.80939227, 0.85793872, 0.90196078, 0.85633803])

In [6]:
rf.fit(X,y)
rf.estimators_

[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=920786178, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=328534659, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features='auto', max_leaf_nodes=None,
                   

## 结合方法

### 对数值型输出
- 平均法
- 加权平均法

## 对分类型输出
- 绝对多数投票法
- 相对多数投票法
- 加权投票法

标准RandomForest的子决策树的参数.

```
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features='auto', max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=813479198, splitter='best'),
```

In [7]:
# 求和有更好的办法
#cvs = []
#for i in range(100):
#    cvs.append(cross_val_score(DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
#                        max_features='auto', max_leaf_nodes=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, presort=False,splitter='best'),X,y,cv=5).mean())
#np.mean(cvs)# 0.6639091719045126

In [8]:
class RandomForest:
    
    def __init__(
        self
        ,T = 30
    ):
        '''
        @params T 循环的次数
        '''
        self.T_ = T
        self.estimators_ = None
        self.X_ = None
        self.y_ = None
        self.es_inbag_sample_args_ = None
        self.es_oobag_sample_args_ = None
        self.es_params_ = {
            'class_weight':None
            ,'criterion':'gini'
            ,'max_depth':6
            ,'max_leaf_nodes':None
            ,'max_features':'auto'
            ,'min_impurity_decrease':0.0
            ,'min_impurity_split':None
            ,'min_samples_leaf':1
            ,'min_samples_split':2
            ,'min_weight_fraction_leaf':0.0
            ,'presort':False
            ,'splitter':'best'
        }#0.6639091719045126
        
    def bootstrap_sampling(self,X,y):
        '''
        自助采样法
        '''
        n = len(y)
        args = np.arange(0,n,1)
        # 放回抽样
        samp_args = np.random.choice(args, n, True)
        uniq_args = np.unique(samp_args) 
        return X[uniq_args] ,y[uniq_args] ,uniq_args

    def __oob_sample_args(self,n,inb_sample_args):
        '''
        获取包外样本索引
        '''
        args = np.arange(0 ,n ,1)
        args[inb_sample_args] = -1
        return args[ args >= 0 ]
        
    def fit(self ,X ,y):
        # 初始化成员变量
        n = len(y)
        self.X_ = X
        self.y_ = y
        self.estimators_ = []
        self.es_inbag_sample_args_ = []
        self.es_oobag_sample_args_ = []
        dt_params = self.es_params_
        for i in range(self.T_):
            # bootstrap采样
            X_sample,y_sample,sample_args = self.bootstrap_sampling(X,y)
            # print(sample_args.shape)
            # 用采样数据训练数据
            rnd_seed = np.random.randint(2**32 - 1)
            clf = DecisionTreeClassifier(
                class_weight=dt_params['class_weight']
                ,criterion=dt_params['criterion']
                ,max_depth=dt_params['max_depth']
                ,max_leaf_nodes=dt_params['max_leaf_nodes']
                ,max_features=dt_params['max_features']
                ,min_impurity_decrease=dt_params['min_impurity_decrease']
                ,min_impurity_split=dt_params['min_impurity_split']
                ,min_samples_leaf=dt_params['min_samples_leaf']
                ,min_samples_split=dt_params['min_samples_split']
                ,min_weight_fraction_leaf=dt_params['min_weight_fraction_leaf']
                ,presort=dt_params['presort']
                ,splitter=dt_params['splitter']
                , random_state=rnd_seed
            ).fit(X_sample,y_sample)
            # 更新成员变量
            self.es_inbag_sample_args_.append(sample_args)
            self.es_oobag_sample_args_.append(self.__oob_sample_args(n, sample_args))
            self.estimators_.append(clf)
        return self
    
    def predict(self,X_test):
        n = len(X_test)
        y_p_set = np.zeros((self.T_,n),dtype=np.int32)
        # For vote
        for i , est in enumerate(self.estimators_):
            yp = est.predict(X_test)
            y_p_set[i]=yp
        # 初始化预测结果
        y_res = np.zeros(n, dtype=np.int32)
        for i in range(n):
            # 对每个样本各个预测器的投票结果
            uniqs, uniq_cnts = np.unique(y_p_set[:,i], return_counts=True)
            # 获取投票数量最大的索引
            cnt_argmax = np.argmax(uniq_cnts)
            #print('uniqs : ',uniqs, 'cnts : ',uniq_cnts,'argmax:',cnt_argmax)
            y_res[i] = uniqs[cnt_argmax]
        return y_res
    
    def __oob_predict(self):
        # 仅考虑未使用x的分类器进行预测.
        
        # 投票的稀疏矩阵,行对应m个样本,列对应投票结果
        y_res = []
        # 用于预测的袋外数据
        y_args = []
        for i , x in enumerate(self.X_):
            # 第i个样本的投票结果
            y_i = []
            # 遍历 T个基分类器
            for t in range(self.T_):
                # 第t个分类器的袋内数据索引.
                es_inbag_args = self.es_inbag_sample_args_[t]
                # 判断当前数据x是否参与了分类器t的训练.
                if np.any(es_inbag_args == i):
                    # 如果已经参与就终结循环,遍历后面的分类器.
                    continue
                # 当前分类没有用到当前数据x = X[i],用当前分类器来预测数据x.
                y_p = self.estimators_[t].predict(x.reshape(1,-1))
                # 记录投票结果
                y_i.append(y_p.ravel()[0])
            # 没有任何分类器没用到当前数据
            if len(y_i) == 0:
                continue
            y_res.append(y_i)
            y_args.append(i)
        n = len(y_args)
        y_final_p = np.full(n, -1)
        
        for i in range(n):
            # 对每个样本各个预测器的投票结果
            uniqs, uniq_cnts = np.unique(np.array(y_res[i]), return_counts=True)
            # 获取投票数量最大的索引
            cnt_argmax = np.argmax(uniq_cnts)
            # print('uniqs : ',uniqs, 'cnts : ',uniq_cnts,'argmax:',cnt_argmax)
            y_final_p[i] = uniqs[cnt_argmax]
        return y_final_p, y_args
    
    def oob_score(self):
        y_p , y_args = self.__oob_predict()
        return np.sum(self.y_[y_args] == y_p) / len(y_args)
    
X,y = load_data()
X_train,X_test,y_train,y_test = train_test_split(X,y)

T=50
# sklearn学习器的训练
sk_rf_clf = RandomForestClassifier(
    n_estimators=T
    , class_weight=None
    , criterion='gini'
    , max_depth=6
    , max_features='auto'
    , max_leaf_nodes=None
    , min_impurity_decrease=0.0
    , min_impurity_split=None
    , min_samples_leaf=1
    , min_samples_split=2
    , min_weight_fraction_leaf=0.0
).fit(X_train,y_train)
sk_rf_score = sk_rf_clf.score(X_test,y_test)
# 我的Simple Bagging 训练
sb = RandomForest(T=T).fit(X_train,y_train)
# 留出法
print('my test score : ' , np.sum(sb.predict(X_test)==y_test) / len(y_test))
print('sklearn rf test score : ' , sk_rf_score)

print("-"*100)
# 自采样oob
sk_rf_clf = RandomForestClassifier(
    n_estimators=T
    , class_weight=None
    , criterion='gini'
    , max_depth=6
    , max_features='auto'
    , max_leaf_nodes=None
    , min_impurity_decrease=0.0
    , min_impurity_split=None
    , min_samples_leaf=1
    , min_samples_split=2
    , min_weight_fraction_leaf=0.0
    , oob_score=True
).fit(X,y)
print('my oob score: ', sb.oob_score())
print('sklearn rf oob score' , sk_rf_clf.oob_score_)

#sb.oob_predict()
#np.argpartition

my test score :  0.9555555555555556
sklearn rf test score :  0.9466666666666667
----------------------------------------------------------------------------------------------------
my oob score:  0.9272457312546399
sklearn rf oob score 0.9360044518642181


In [9]:
X,y = load_data()
X_train,X_test,y_train,y_test = train_test_split(X,y)

dt_params = {
    'class_weight':None
    ,'criterion':'gini'
    ,'max_depth':6
    ,'max_leaf_nodes':None
    ,'max_features':'auto'
    ,'min_impurity_decrease':0.0
    ,'min_impurity_split':None
    ,'min_samples_leaf':1
    ,'min_samples_split':2
    ,'min_weight_fraction_leaf':0.0
    ,'presort':False
    ,'splitter':'best'
}
dt_scores=[]
for i in range(50):
    dt_clf = DecisionTreeClassifier(
                    class_weight=dt_params['class_weight']
                    ,criterion=dt_params['criterion']
                    ,max_depth=dt_params['max_depth']
                    ,max_leaf_nodes=dt_params['max_leaf_nodes']
                    ,max_features=dt_params['max_features']
                    ,min_impurity_decrease=dt_params['min_impurity_decrease']
                    ,min_impurity_split=dt_params['min_impurity_split']
                    ,min_samples_leaf=dt_params['min_samples_leaf']
                    ,min_samples_split=dt_params['min_samples_split']
                    ,min_weight_fraction_leaf=dt_params['min_weight_fraction_leaf']
                    ,presort=dt_params['presort']
                    ,splitter=dt_params['splitter']
                )
    dt_clf.fit(X_train,y_train)
    dt_scores.append(dt_clf.score(X_test,y_test))
print('DT mean scores : ',np.mean(dt_scores))

DT mean scores :  0.7186666666666668
